### 카테고리별 지원 파싱 (생계지원)
생계지원 (22~55페이지) 파싱

In [1]:
import os
import json
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
import fitz

load_dotenv()

True

In [2]:
class TableData(BaseModel):
    columns: list = Field(description="표의 열 제목들")
    rows: list = Field(description="표의 행 데이터들")

class WelfareProgram(BaseModel):
    title: str = Field(description="복지 프로그램의 정확한 지원명")
    target: str = Field(description="지원 대상자")
    content: str = Field(description="지원 내용")
    method: str = Field(description="신청 방법")
    inquiry: str = Field(description="문의처")
    tables: list[TableData] = Field(description="표 데이터 구조")

In [3]:
def extract_page_text(pdf_path, page_num):
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_num - 1)
    text = page.get_text("text")
    doc.close()
    return text

In [4]:
def parse_livelihood_support(text, page_num):
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0,
        request_timeout=60,
        max_retries=3
    )
    
    parser = JsonOutputParser(pydantic_object=WelfareProgram)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """생계지원 문서 전용 파서입니다.

검증 로직:
1. "대상", "내용", "방법", "문의" 4개 필수 키워드 존재 확인
2. 키워드 중 하나라도 없으면 null 반환

제도명 추출 규칙:
- "나에게 힘이 되는 복지서비스"는 페이지 헤더이므로 제외
- "생계 지원"은 카테고리명이므로 제외
- 실제 제도명만 추출 (예: "기초생활보장제도", "차상위계층 지원", "긴급복지지원")

표 처리 규칙:
1. content: 표 내용을 자연어로 요약하여 포함
2. tables: 표가 있으면 구조화된 데이터로 저장
   - columns: 실제 표의 열 제목을 정확히 추출 (예: ["가구규모", "생계급여", "의료급여"])
   - rows: 실제 표의 행 데이터를 정확히 추출 (예: [["1인", "76만원", "1종"]])
   - 표의 실제 구조를 그대로 반영하여 저장
3. 표가 없으면 tables는 빈 배열

추출 최적화:
- 제도명: 실제 생계지원제도명만
- 대상: 소득기준, 재산기준, 가구구성 등 구체적 조건
- 내용: 지원금액, 지원기간, 지원내용 (표 데이터 요약 포함)
- 방법: 신청기관, 온라인/오프라인 구분, 필요서류
- 문의: 콜센터, 웹사이트, 담당기관
- tables: 표가 있으면 실제 컬럼명과 데이터로 구조화하여 저장

{format_instructions}"""),

        ("human", """생계지원 문서를 분석합니다.

1단계: 필수 키워드 확인
- "대상", "내용", "방법", "문의" 4개 키워드 모두 존재하는지 확인
- 하나라도 없으면 null 반환

2단계: 제도명 추출
- "나에게 힘이 되는 복지서비스" 제외
- "생계 지원" 제외
- 실제 생계지원제도명만 추출

3단계: 정보 추출
- content: 표 내용을 자연어로 요약하여 포함
- tables: 표가 있으면 실제 컬럼명과 행 데이터를 정확히 추출하여 저장
- 각 키워드 바로 다음 내용을 정확히 추출

텍스트:
{text}
""")
    ])
    
    chain = prompt | llm | parser
    
    for attempt in range(3):
        try:
            result = chain.invoke({
                "text": text,
                "format_instructions": parser.get_format_instructions()
            })
            
            if result is None:
                print(f"페이지 {page_num}: 필수 키워드 부족으로 건너뛰기")
                return None
            
            result["page"] = page_num
            result["category"] = "생계지원"
            
            return result
        except Exception as e:
            print(f"파싱 시도 {attempt + 1}/3 실패 (페이지 {page_num}): {e}")
            if attempt < 2:
                time.sleep(2)
            else:
                return None

In [5]:
# 실행
pdf_path = "../../data/pdfs/[통합]+2025+나에게+힘이+되는+복지서비스.pdf"
target_pages = list(range(22, 56))  # 생계지원 섹션 (22~55페이지)
livelihood_programs = []

for i, page_num in enumerate(target_pages):
    print(f"페이지 {page_num} 파싱 중... ({i+1}/{len(target_pages)})")
    
    page_text = extract_page_text(pdf_path, page_num)
    program = parse_livelihood_support(page_text, page_num)
    
    if program:
        livelihood_programs.append(program)
        table_count = len(program.get('tables', []))
        print(f"{program['title']} 추출 완료 (표 {table_count}개)")
    else:
        print(f"페이지 {page_num} 건너뛰기")
    
    if i < len(target_pages) - 1:
        time.sleep(1)

페이지 22 파싱 중... (1/34)
기초생활보장제도 추출 완료 (표 1개)
페이지 23 파싱 중... (2/34)
부채･신용정보개선 추출 완료 (표 0개)
페이지 24 파싱 중... (3/34)
기초생활수급자를 위한 요금감면제도 추출 완료 (표 1개)
페이지 25 파싱 중... (4/34)
긴급복지 지원제도 추출 완료 (표 0개)
페이지 26 파싱 중... (5/34)
긴급복지지원 추출 완료 (표 1개)
페이지 27 파싱 중... (6/34)
온가족보듬사업 추출 완료 (표 1개)
페이지 28 파싱 중... (7/34)
긴급돌봄 지원사업 추출 완료 (표 1개)
페이지 29 파싱 중... (8/34)
생계 지원 추출 완료 (표 1개)
페이지 30 파싱 중... (9/34)
상병수당 지원 추출 완료 (표 1개)
페이지 31 파싱 중... (10/34)
생계지원 추출 완료 (표 0개)
페이지 32 파싱 중... (11/34)
행복주택 공급 추출 완료 (표 0개)
페이지 33 파싱 중... (12/34)
행복주택 생계지원 추출 완료 (표 0개)
페이지 34 파싱 중... (13/34)
분양전환 공공임대주택 공급 추출 완료 (표 0개)
페이지 35 파싱 중... (14/34)
공공주택 분양전환 지원 추출 완료 (표 1개)
페이지 36 파싱 중... (15/34)
매입임대주택 지원 추출 완료 (표 0개)
페이지 37 파싱 중... (16/34)
다가구주택･다세대·연립주택 매입임대 추출 완료 (표 1개)
페이지 38 파싱 중... (17/34)
전세임대 지원 프로그램 추출 완료 (표 1개)
페이지 39 파싱 중... (18/34)
전세임대 지원 프로그램 추출 완료 (표 1개)
페이지 40 파싱 중... (19/34)
주거취약계층 주거지원 추출 완료 (표 0개)
페이지 41 파싱 중... (20/34)
긴급 주거지원 추출 완료 (표 0개)
페이지 42 파싱 중... (21/34)
주거급여(맞춤형급여) 추출 완료 (표 1개)
페이지 43 파싱 중... (22/34)
슬레이트

In [6]:
# 결과 저장
os.makedirs("outputs", exist_ok=True)
output_path = "../../data/outputs/생계지원_2025.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(livelihood_programs, f, ensure_ascii=False, indent=2)

print(f"\n파싱 완료: {len(livelihood_programs)}개 프로그램")
print(f"저장 위치: {output_path}")


파싱 완료: 34개 프로그램
저장 위치: ../../data/outputs/생계지원_2025.json


In [7]:
# 결과 미리보기
for program in livelihood_programs:
    table_count = len(program.get('tables', []))
    print(f"\n{program['title']} (페이지 {program['page']}, 표 {table_count}개)")
    print(f"대상: {program['target'][:80]}...")
    print(f"내용: {program['content'][:80]}...")
    if table_count > 0:
        print(f"표: {program['tables'][0]['columns'][:3]}...")


기초생활보장제도 (페이지 22, 표 1개)
대상: 소득인정액 기준, 부양의무자 기준을 충족하는 저소득층 수급권자...
내용: 국민기초생활보장법에 따른 국민기초생활보장제도는 소득 및 부양의무자 기준을 충족하는 저소득층에게 생계, 의료, 주거, 교육급여를 통해 최저 생활을...
표: ['구분', '1차(의원)', '2차(병원)']...

부채･신용정보개선 (페이지 23, 표 0개)
대상: 생계급여, 의료급여, 주거급여 수급자...
내용: 해산 급여: 출산(예정)한 경우 아이 1명당 70만 원(쌍둥이는 140만 원) 지급. 장제 급여: 사망한 경우 장례를 치르는 사람에게 사망자 1...

기초생활수급자를 위한 요금감면제도 (페이지 24, 표 1개)
대상: 생계급여, 의료급여 수급자 / 주거급여, 교육급여 수급자 / 각종 공공요금 수급자...
내용: 기초생활수급자를 위한 요금감면제도는 통신요금, 주거급여·교육급여, 각종 공공요금 등 다양한 분야에서 지원을 제공합니다. 통신요금 감면은 유선전화...
표: ['지원내용', '생계급여·의료급여', '주거급여·교육급여']...

긴급복지 지원제도 (페이지 25, 표 0개)
대상: 생계유지가 곤란한 저소득층으로 아래 위기사유와 소득･재산 기준을 충족하는 가구...
내용: 긴급복지 지원제도는 주요 위기 상황으로 인해 생계유지가 어려운 분들에게 생활비, 의료비, 주거비 등을 지원하는 제도입니다. 지원 대상은 주요 위...

긴급복지지원 (페이지 26, 표 1개)
대상: 긴급적인 생계비, 의료비, 주거비, 교육비 등을 필요로 하는 가구...
내용: 긴급적인 생계비, 의료비, 주거비, 교육비 등을 지원합니다. 생계유지비, 의료, 주거, 사회복지, 교육, 연료비, 해산비, 장제비, 전기요금 등...
표: ['구분', '지원내용', '지원금액']...

온가족보듬사업 (페이지 27, 표 1개)
대상: 한부모가족, 다문화가족, 1인 가구, 노부모 부양가족, 손자녀 돌봄 조부모, 청소년(한)부모 등 가족기능 회복 및 역량 강화를 희망하는 